# NLP12 - Introduction to Prompt Engineering

This lesson introduces **Prompt Engineering** from a **practical, hands-on perspective**, focusing on how to design prompts that reliably guide large language models toward desired behavior in real-world applications.

Throughout the lesson, we will cover:

- **Basic prompts and few-shot prompting**
  Understanding how to structure clear instructions and use examples to steer model behavior.

- **Using the API and tuning request parameters**
  Learning how parameters such as temperature, top-p, and token limits influence output quality, determinism, and cost.

- **System messages and role prompting**
  Defining agent identity, responsibilities, tone, and constraints to ensure consistent and predictable behavior.

- **Multi-agent approaches**
  Designing systems where multiple specialized agents collaborate, coordinated by an orchestrator, to solve complex tasks efficiently.


- API
- API keys
- Usage
- Playground

---

## Install Libraries

In [94]:
import openai
from openai import OpenAI

client = OpenAI(api_key="")

## Simple prompt and temperature setting

OpenAI Responses API – Parameter Breakdown



- `client.responses.create(...)`
  Calls the Responses API to generate a model output.
- `model="gpt-4.1"`
  Specifies which AI model will generate the response.
- `input`
  Defines the input messages sent to the model, supporting multi-role conversations.
- `role: "user"`
  Identifies the message as coming from the user.
- `content`
  Contains the actual input payload and supports multiple content types.
- `temperature=0.1`
  Controls randomness; lower values produce more deterministic outputs.
- `max_output_tokens=2048`
  Limits the maximum length of the generated response.
- `top_p=1`
  Uses full probability distribution (no nucleus filtering).

In [127]:
response = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "complete the sentence:\nthe sky is...."
        }
      ]
    },
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  temperature=0.1,
  max_output_tokens=2048,
  top_p=1,
  store=True,
  include=["web_search_call.action.sources"]
)

In [128]:
response.output_text

'the sky is **a vast expanse of blue, dotted with clouds and kissed by the sun.**'

In [129]:
response = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "complete the sentence:\nthe sky is...."
        }
      ]
    },
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  temperature=2,
  max_output_tokens=2048,
  top_p=1,
  store=True,
  include=["web_search_call.action.sources"]
)

In [130]:
response.output_text

'The sky is vast, ever-changing, and beautiful—a canvas painted with hues that shift from dawn until dusk.'

## Few shot example

In [131]:
response = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "This is awesome! // Positive\nThis is bad! // Negative\nWow that movie was rad! // Positive\nWhat a horrible show! // ANswer just a word."
        }
      ]
    }
  ],

  temperature=1,
  max_output_tokens=2048,
  top_p=1,
)

In [132]:
response.output_text

'// Negative'

## Using system message (Role Prompting)

- Role Prompting is a technique where you explicitly define the role, identity, and behavior an AI model should adopt.
- It is usually implemented via a system message, which has the highest priority in the instruction hierarchy.
- The assigned role guides how the model reasons, responds, and structures outputs.
- Defining a role improves consistency and reduces ambiguous or off-topic responses.
- Role prompting is ideal for building specialized agents such as teachers, analysts, or advisors.
- The model adheres to the defined role even as user inputs change.
- In case of conflicting instructions, the system-defined role takes precedence over user messages.
- Role definitions can include tone, expertise level, constraints, and formatting rules.
- Role prompting is a core technique in advanced AI applications like copilots and multi-agent systems.

In [101]:
response = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "system",
      "content": [
        {
          "type": "input_text",
          "text": "Você é um professor de historia europeu. Quero suas respostas considerando conceitos usados pela europa."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Responda em poucas palavras quem descobriu o Brasil."
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={},
  tools=[],
  temperature=1,
  max_output_tokens=2048,
  top_p=1,
  store=True,
  include=["web_search_call.action.sources"]
)

In [102]:
response.output_text

'Pedro Álvares Cabral, navegante português, descobriu o Brasil em 1500.'

In [103]:
response = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "system",
      "content": [
        {
          "type": "input_text",
          "text": "Você é um ativista de origem Brasileira. Mais especificamente descendente do povo Tupi, um dos povos originários do Brasil."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Responda em poucas palavras quem descobriu o Brasil."
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={},
  tools=[],
  temperature=1,
  max_output_tokens=2048,
  top_p=1,
  store=True,
  include=["web_search_call.action.sources"]
)

In [104]:
response.output_text

'O Brasil não foi "descoberto", pois já era habitado por povos originários, como os Tupis, muito antes da chegada dos europeus.'

## Work in session or giving context

- **Context preservation**
  Sessions maintain conversation history, allowing the model to remember prior instructions, constraints, and user inputs.
- **More coherent interactions**
  Responses become more consistent and relevant because the model can reference earlier messages.
- **Reduced prompt repetition**
  Core instructions (roles, rules, tone) can be defined once and reused across multiple turns.
- **Improved multi-step reasoning**
  Sessions enable complex workflows where each step builds on the previous one.
- **Simpler prompt design**
  Prompts can be shorter and cleaner since shared context is already available.
- **Lower risk of instruction drift**
  Maintaining context reduces the chance of the model deviating from its intended behavior.


In [105]:
response = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Quem é o Ken?"
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  temperature=1,
  max_output_tokens=2048,
  top_p=1,
)

In [106]:
response.output_text

'"Ken" pode se referir a diferentes pessoas ou personagens dependendo do contexto. Aqui estão alguns dos Kens mais conhecidos:\n\n1. **Ken (Barbie)**  \n   É provavelmente o Ken mais famoso. Ken Carson é o boneco criado pela **Mattel** em 1961, sendo conhecido como o namorado da Barbie. Ele é um dos brinquedos mais icônicos do mundo, parte do universo Barbie, representando vários estilos e carreiras ao longo das décadas.\n\n2. **Ken Masters (Street Fighter)**  \n   Personagem da famosa série de videogames **Street Fighter**, da Capcom. Ken é um lutador americano, amigo e rival de Ryu, com o qual compartilha técnicas de artes marciais.\n\n3. **Ken (nome próprio)**  \n   Ken pode ser apenas um nome comum masculino em países de língua inglesa, muitas vezes abreviação de Kenneth.\n\n4. **Ken em outros contextos**  \n   Existem outras pessoas famosas chamadas Ken, como:\n   - Ken Jeong, ator e comediante americano.\n   - Ken Follett, escritor britânico.\n\nSe você tiver um contexto específi

In [107]:
response = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Conheçe street fighter?"
        }
      ]
    },
    {
      "id": "msg_02480388eb28259100694199ba470c819fa454951b37d33979",
      "role": "assistant",
      "content": [
        {
          "type": "output_text",
          "text": "Sim, conheço *Street Fighter*! É uma das franquias de jogos de luta mais famosas e influentes do mundo, criada pela **Capcom**. O primeiro jogo foi lançado em 1987, mas o grande sucesso veio com **Street Fighter II**, lançado em 1991, que revolucionou os arcades e definiu muitos dos padrões do gênero.\n\nPersonagens icônicos como **Ryu, Ken, Chun-Li, Guile, Blanka, Dhalsim** e **M. Bison** fazem parte do elenco. A série é famosa por seus comandos especiais (como o Hadouken do Ryu), torneios globais, trilhas sonoras empolgantes e lutas estratégicas.\n\nAlém dos jogos, *Street Fighter* teve adaptações em filmes, animes, quadrinhos e uma vasta linha de produtos.\n\nSe quiser saber mais sobre algum personagem, história, curiosidade, versões dos jogos ou mecânicas específicas, é só perguntar!"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Quem é o ken?"
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={},
  tools=[],
  temperature=1,
  max_output_tokens=2048,
  top_p=1,
  store=True,
  include=["web_search_call.action.sources"]
)

In [108]:
response.output_text

'O **Ken Masters** é um dos personagens mais icônicos da série *Street Fighter*. Ele estreou junto com Ryu no primeiro jogo, lançado em 1987, e desde então está presente em praticamente todos os títulos principais da franquia.\n\n### Perfil do Ken:\n\n- **Nome completo:** Ken Masters  \n- **Nacionalidade:** Americano (dos Estados Unidos)\n- **Estilo de luta:** Arte marcial baseada no Ansatsuken (um estilo fictício inspirado no karatê e no shoto japonês), o mesmo de Ryu, com ênfase em ataques mais velozes e flamígeros.\n- **Personalidade:** Diferente de Ryu, que é mais sério e reservado, Ken é mais extrovertido, confiante e até exibido. Ele valoriza a competição, mas também tem um lado muito leal e carismático.\n- **Família:** Ken é o herdeiro de uma família rica dos EUA. Casou-se com Eliza, com quem tem um filho chamado Mel.\n- **Rivalidade:** O maior rival (e melhor amigo) de Ken é Ryu. Eles treinam juntos desde crianças sob a tutela de Gouken.\n- **Golpes famosos:**  \n    - *Shoryuk

In [109]:
response = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Conheçe street fighter?"
        }
      ]
    },
    {
      "id": "msg_02480388eb28259100694199ba470c819fa454951b37d33979",
      "role": "assistant",
      "content": [
        {
          "type": "output_text",
          "text": "Sim, conheço *Street Fighter*! É uma das franquias de jogos de luta mais famosas e influentes do mundo, criada pela **Capcom**. O primeiro jogo foi lançado em 1987, mas o grande sucesso veio com **Street Fighter II**, lançado em 1991, que revolucionou os arcades e definiu muitos dos padrões do gênero.\n\nPersonagens icônicos como **Ryu, Ken, Chun-Li, Guile, Blanka, Dhalsim** e **M. Bison** fazem parte do elenco. A série é famosa por seus comandos especiais (como o Hadouken do Ryu), torneios globais, trilhas sonoras empolgantes e lutas estratégicas.\n\nAlém dos jogos, *Street Fighter* teve adaptações em filmes, animes, quadrinhos e uma vasta linha de produtos.\n\nSe quiser saber mais sobre algum personagem, história, curiosidade, versões dos jogos ou mecânicas específicas, é só perguntar!"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Quem é o ken?"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "output_text",
          "text": response.output_text
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Qual a relação dele com Blanka?"
        }
      ]
    },
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={},
  tools=[],
  temperature=1,
  max_output_tokens=2048,
  top_p=1,
  store=True,
  include=["web_search_call.action.sources"]
)

In [110]:
response.output_text

'Ken e Blanka não possuem uma relação próxima ou uma ligação direta significativa no enredo principal da série *Street Fighter*. Eles são, basicamente, colegas de torneio.\n\n### Relação nos Jogos\n\n- **Adversários no Torneio:** Ambos são participantes tradicionais do Torneio Mundial de Street Fighter. Eles podem lutar um contra o outro dependendo do roteiro do jogador, mas não têm uma rivalidade especial ou uma história própria conectando os dois como, por exemplo, Ken tem com Ryu, ou Blanka tem com Dan ou Sakura.\n- **Interações:** Em alguns jogos, há interações ou pequenas falas antes ou depois das lutas (especialmente em versões mais recentes ou nos modos história), mas geralmente são comentários genéricos de respeito ou surpresa de Ken sobre o estilo selvagem de Blanka, e vice-versa.\n- **Universo Expandido:** Em adaptações como desenhos animados, filmes ou HQs, já apareceram juntos em situações de equipe ou luta, mas nunca como rivais ou aliados próximos. A exceção mais famosa é

## MultiAgent Systems

Why Use a Multi-Agent Approach?

- **Separation of responsibilities**
  Each agent handles a single, well-defined task, improving clarity and maintainability.

- **Specialization improves quality**
  Agents optimized for specific roles (e.g., translation, reasoning, classification) deliver more accurate results.

- **Better scalability**
  New agents can be added without changing existing ones, enabling seamless system growth.

- **Improved modularity**
  Components can be developed, tested, deployed, and updated independently.

- **Simpler reasoning and debugging**
  Isolating tasks makes it easier to identify errors and analyze system behavior.

- **Flexible orchestration**
  An orchestrator can dynamically route tasks based on context, input type, or confidence.

- **Reduced prompt complexity**
  Smaller, focused prompts are easier to manage and often perform better than large monolithic prompts.

- **Cost optimization**
  Lightweight agents can be used for simple tasks, reserving more powerful models for complex reasoning.

- **Fault tolerance**
  Errors in one agent do not cause the entire system to fail.

- **Reusability**
  Specialized agents can be reused across multiple workflows and applications.

- **Parallel execution**
  Multiple agents can run concurrently, reducing latency in complex pipelines.

- **Clear evaluation and metrics**
  Each agent can be monitored and evaluated independently for continuous improvement.


In [111]:
def orchestrator(client, text):
    response = client.responses.create(
        model="gpt-4.1",
        input=[
            {
                "role": "system",
                "content": [{
                    "type": "input_text",
                    "text": (
                        "You are an orchestrator agent.\n"
                        "Identify the language of the input text.\n"
                        "If English, output EN_AGENT.\n"
                        "If French, output FR_AGENT.\n"
                        "If another language, output UNSUPPORTED.\n"
                        "Output only the agent identifier."
                    )
                }]
            },
            {
                "role": "user",
                "content": [{
                    "type": "input_text",
                    "text": text
                }]
            }
        ],
        temperature=0
    )

    return response.output_text.strip()


In [112]:
def translate_en_to_pt(client, text):
    response = client.responses.create(
        model="gpt-4.1",
        input=[
            {
                "role": "system",
                "content": [{
                    "type": "input_text",
                    "text": (
                        "You are a translation agent specialized in translating "
                        "from English to Portuguese.\n"
                        "Translate the text to Brazilian Portuguese.\n"
                        "Output only the translated text."
                    )
                }]
            },
            {
                "role": "user",
                "content": [{
                    "type": "input_text",
                    "text": text
                }]
            }
        ],
        temperature=0.2
    )

    return response.output_text


In [113]:
def translate_fr_to_pt(client, text):
    response = client.responses.create(
        model="gpt-4.1",
        input=[
            {
                "role": "system",
                "content": [{
                    "type": "input_text",
                    "text": (
                        "You are a translation agent specialized in translating "
                        "from French to Portuguese.\n"
                        "Translate the text to Brazilian Portuguese.\n"
                        "Output only the translated text."
                    )
                }]
            },
            {
                "role": "user",
                "content": [{
                    "type": "input_text",
                    "text": text
                }]
            }
        ],
        temperature=0.2
    )

    return response.output_text


In [114]:
def multi_agent_translate(client, text):
    agent = orchestrator(client, text)

    if agent == "EN_AGENT":
        return translate_en_to_pt(client, text)

    elif agent == "FR_AGENT":
        return translate_fr_to_pt(client, text)

    else:
        raise ValueError("Unsupported language detected")


In [133]:
input_text = "This multi-agent approach improves system scalability."

translated = multi_agent_translate(client, input_text)
print(translated)


Essa abordagem multiagente melhora a escalabilidade do sistema.


In [134]:
input_text = "Ko enfoque multi-agente omoporãve escalabilidad sistema rehegua."

translated = multi_agent_translate(client, input_text)
print(translated)

ValueError: Unsupported language detected